In [1]:
import nltk
from nltk.tokenize import sent_tokenize
nltk.download('stopwords')
from nltk.corpus import stopwords
with open('Disease.txt', 'r') as f:
    file = f.readlines()
    text = "".join(file)


 










[nltk_data] Downloading package stopwords to
[nltk_data]     /home/sharbadeepa/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [2]:
import spacy
nlp = spacy.load("en_core_web_sm")
doc = nlp(text)
ner = nlp.get_pipe('ner')
print(ner)
print(nlp.pipe_names)


['tok2vec', 'tagger', 'parser', 'attribute_ruler', 'lemmatizer', 'ner']


# Using Regex finding out the index of the diseases

In [3]:
import spacy
from spacy import displacy
import re
string = text
import requests
from nltk.corpus import stopwords
from bs4 import BeautifulSoup
url_link = 'https://www.nhsinform.scot/illnesses-and-conditions/a-to-z#A'
request = requests.get(url_link)
Soup = BeautifulSoup(request.text, 'lxml')
stop_words = set(stopwords.words('english'))
diseases = []
heading_tags = ["h1", "h2", "h3"]
for tags in Soup.find_all(heading_tags):
    diseases.append(tags.text.strip())

Train_data=[]
for disease in diseases:
    if disease not in stop_words and len(disease) > 3:
        match = re.search(disease, string)
        if match:
            match_dis = [match.start(), match.end(),'DISEASES']
            new = [match.start()]
            Train_data.append(tuple(match_dis))
Train_data = [(string, {'entities':Train_data})]

for _,tags in Train_data:
    for ent in tags.get('entities'):
        ner.add_label(ent[2])

string[3046:30478]

'g most forms of cancer, heart disease, and genetic disease.\nAcquired disease\nAn acquired disease is one that began at some point during one\'s lifetime, as opposed to disease that was already present at birth, which is congenital disease. Acquired sounds like it could mean "caught via contagion", but it simply means acquired sometime after birth. It also sounds like it could imply secondary disease, but acquired disease can be primary disease.\nAcute disease\nAn acute disease is one of a short-term nature (acute); the term sometimes also connotes a fulminant nature\nChronic condition or chronic disease\nA chronic disease is one that persists over time, often characterized as at least six months but may also include illnesses that are expected to last for the entirety of one\'s natural life.\nCongenital disorder or congenital disease\nA congenital disorder is one that is present at birth. It is often a genetic disease or disorder and can be inherited. It can also be the result of a v

# Using Displacy for the visuvalisation

In [4]:
import random
from spacy.util import minibatch,compounding
from spacy.training import Example
from spacy.tokens import Span


In [5]:
pipe_exceptions = ['ner', 'trf_wordpiecer', 'tfg_tok2vec']
unaffected_pipes = [pipe for pipe in nlp.pipe_names if pipe not in pipe_exceptions]
print(unaffected_pipes)


['tok2vec', 'tagger', 'parser', 'attribute_ruler', 'lemmatizer']


In [6]:
 with nlp.disable_pipes(*unaffected_pipes):
        sizes = compounding(1.0, 4.0, 1.001)
        for itn in range(100):

            random.shuffle(Train_data)
            batches = minibatch(Train_data, size=sizes)
            losses = {}

            for batch in batches:
                for text,annotations in batch:
                    doc = nlp.make_doc(text)
                    example = Example.from_dict(doc, annotations)
                    
                    nlp.update([example], drop=0.5, losses=losses)
                    print("losses", losses)



losses {'ner': 342.546132594347}
losses {'ner': 223.73317213106202}
losses {'ner': 134.08578735344418}
losses {'ner': 97.57241746829823}
losses {'ner': 53.35849000370945}
losses {'ner': 45.74790943169501}
losses {'ner': 25.802718441917904}
losses {'ner': 14.554322024196448}
losses {'ner': 13.923905563550342}
losses {'ner': 11.215902467145856}
losses {'ner': 7.3783691794084785}
losses {'ner': 6.851142305999891}
losses {'ner': 6.8061492188094315}
losses {'ner': 6.701104550163279}
losses {'ner': 7.659260383823104}
losses {'ner': 8.02782605884586}
losses {'ner': 6.166576505481647}
losses {'ner': 6.03061375990244}
losses {'ner': 5.997632372866976}
losses {'ner': 6.897326168533482}
losses {'ner': 5.676165911931491}
losses {'ner': 5.958266820185315}
losses {'ner': 6.016922666647133}
losses {'ner': 5.164052106685631}
losses {'ner': 6.31843481967735}
losses {'ner': 5.032448314286739}
losses {'ner': 5.385313273924112}
losses {'ner': 5.7497144647986715}
losses {'ner': 4.8981866958756655}
losses {

In [7]:
doc = nlp('Scanning Migraine micrograph of Epilepsy, a bacterium that causes tuberculosis')
print("Entities", [(ent.text, ent.label_) for ent in doc.ents])

Entities [('Migraine', 'DISEASES'), ('Epilepsy', 'DISEASES')]


# Performing Trigrams


[]